# Simulation template 

In this notebook we run the machine learning analysis of topological phase transitions occurring  in both nearest-neighbours SSH models (ssh1) and second neighbours models (ssh2) as decribed in the paper [Machine learning topological phases in real space](https://arxiv.org/abs/1901.01963). Here the simulation is run with features generated from fourier components in the first scenario. This scenario is characterized by using only the EVEN wavevector space eigenmodes, computed from ALL real space components.

## Defining parameters

In [1]:
%%time
%load_ext autoreload
%autoreload 2
from simulation import *

CPU times: user 756 ms, sys: 218 ms, total: 974 ms
Wall time: 616 ms


In [2]:
%%time
### Dataset and simulation parameters
#csv_path = SSH2_PERIODIC_100_6561_CSV 
csv_path = SSH1_PERIODIC_100_6561_CSV 
model_name = "DecisionTreeClassifier"
model_kw = {"criterion":"entropy"}
allowed_windings = [0,1]
#allowed_windings = [-1,0,1,2]
simulation_dir = SSH1_REAL_SPACE_ALL_SITES
#simulation_dir = SSH2_PERIODIC_ENGINEERED_100_6561_SIMULATION_DIR
val_split = 0.9  
features_to_use = None
shuffle_features = False
random_state = 763

### Fourier features
fourier_mode = "dst"
fourier_features_to_use = None
fourier_real = "imag"
fourier_normalize = False
fourier_fillna = False

### Running a simulation
n_experiments = 5
start_n = 0
fit_params = None
#mode=mode
shuffle_rows = True
pred_params = None
random_features = False
######### DON'T SET THIS TO TRUE UNLESS YOUR DATASET IS SMALL!! WILL FLOOD YOUR MEMORY!!!
store_in_lists = False   
########## BELOW ARE THE PARAMETERS THAT CONTROL WHAT WILL BE SAVED
save_eigenvector=True
save_hamiltonian=True 
save_accuracy=True 
save_models=True

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 5.25 µs


In [3]:
%%time
#Starting an instance of Simulation with a decision tree
model = DecisionTreeClassifier(criterion="entropy")
#dict_args = {"csv_path":csv_path, "model":model, "allowed_windings":allowed_windings,\
#             "simulation_dir":simulation_dir, "val_split":val_split, "features_to_use":features_to_use,\
#            "shuffle_features":shuffle_features, "random_state":random_state}
simulation = Simulation(csv_path,model_name,model_kw,allowed_windings,simulation_dir,val_split,features_to_use,\
                       shuffle_features,fourier_mode,fourier_features_to_use,fourier_real,\
                       fourier_normalize, fourier_fillna , random_state)
#simulation = Simulation(**dict_args)


#(self, csv_path, model_name, model_kw, allowed_windings, simulation_dir = None, val_split = 0, 
# features_to_use = None, shuffle_features = False, fourier_mode=None, fourier_features_to_use=None,
# fourier_real = False, fourier_normalize = False, Fourier_fillna=False, random_state = None):



print("Info on all data: \n")
simulation.dataframe.info()
simulation.dataframe.head()

Info on all data: 

<class 'pandas.core.frame.DataFrame'>
Int64Index: 656100 entries, 0 to 656099
Columns: 108 entries, id to feat99
dtypes: float64(103), int32(3), object(2)
memory usage: 538.1+ MB
CPU times: user 6.37 s, sys: 679 ms, total: 7.04 s
Wall time: 6.23 s


,id,path,t1,t2,winding,phase,pred_phase,type_of,feat0,feat1,...,feat90,feat91,feat92,feat93,feat94,feat95,feat96,feat97,feat98,feat99
0,0,H_-2_-2_0.509296.dat,-2.0,-2.0,0.509296,999,666,test,0.100000,0.100000,...,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000
1,0,H_-2_-2_0.509296.dat,-2.0,-2.0,0.509296,999,666,test,0.141421,0.141142,...,0.114412,0.108967,0.123928,0.119406,0.131490,0.127962,0.136978,0.134500,0.140306,0.138916
2,0,H_-2_-2_0.509296.dat,-2.0,-2.0,0.509296,999,666,test,0.141421,0.140306,...,0.043702,0.026500,0.075777,0.060214,0.103092,0.090145,0.123928,0.114412,0.136978,0.131490
3,0,H_-2_-2_0.509296.dat,-2.0,-2.0,0.509296,999,666,test,-0.100000,0.100000,...,-0.100000,0.100000,-0.100000,0.100000,-0.100000,0.100000,-0.100000,0.100000,-0.100000,0.100000
4,0,H_-2_-2_0.509296.dat,-2.0,-2.0,0.509296,999,666,test,0.000509,-0.008372,...,-0.082713,-0.089753,-0.067684,-0.075347,-0.051587,-0.059753,-0.034677,-0.043217,-0.017220,-0.026000


####  Fourier features

In [4]:
%%time
#simulation.generate_fourier_df()
if simulation.fourier_mode is not None:
    simulation.fourier_dataframe.info() 
    simulation.fourier_dataframe.sample(15)


<class 'pandas.core.frame.DataFrame'>
Int64Index: 656100 entries, 0 to 656099
Data columns (total 57 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   id          656100 non-null  int32  
 1   path        656100 non-null  object 
 2   t1          656100 non-null  float64
 3   t2          656100 non-null  float64
 4   winding     656100 non-null  float64
 5   phase       656100 non-null  int32  
 6   pred_phase  656100 non-null  int32  
 7   type_of     656100 non-null  object 
 8   dst_feat0   656100 non-null  float64
 9   dst_feat1   656100 non-null  float64
 10  dst_feat2   656100 non-null  float64
 11  dst_feat3   656100 non-null  float64
 12  dst_feat4   656100 non-null  float64
 13  dst_feat5   656100 non-null  float64
 14  dst_feat6   656100 non-null  float64
 15  dst_feat7   656100 non-null  float64
 16  dst_feat8   656100 non-null  float64
 17  dst_feat9   656100 non-null  float64
 18  dst_feat10  656100 non-null  float64
 19  ds

In [5]:
print("features_to_use: ")
print("\n")
print(simulation.features_to_use)
if simulation.features_to_use is not None:
    print("Number of real space features:")
    print("\n")
    print(len(simulation.features_to_use))
print("\n")
print("fourier_features: ")
print("\n")
print(simulation.fourier_features)
print("\n")
if simulation.fourier_features is not None:
    print("Number of Fourier features: ")
    print("\n")
    print(len(simulation.fourier_features))
print("\n")
print("Fourier operator: ")
print(simulation.fourier_operator)
print("\n")
if simulation.fourier_operator is not None:
    print("Shape of Fourier operator: ")
    print(simulation.fourier_operator.shape)

features_to_use: 


None


fourier_features: 


['dst_feat0', 'dst_feat1', 'dst_feat2', 'dst_feat3', 'dst_feat4', 'dst_feat5', 'dst_feat6', 'dst_feat7', 'dst_feat8', 'dst_feat9', 'dst_feat10', 'dst_feat11', 'dst_feat12', 'dst_feat13', 'dst_feat14', 'dst_feat15', 'dst_feat16', 'dst_feat17', 'dst_feat18', 'dst_feat19', 'dst_feat20', 'dst_feat21', 'dst_feat22', 'dst_feat23', 'dst_feat24', 'dst_feat25', 'dst_feat26', 'dst_feat27', 'dst_feat28', 'dst_feat29', 'dst_feat30', 'dst_feat31', 'dst_feat32', 'dst_feat33', 'dst_feat34', 'dst_feat35', 'dst_feat36', 'dst_feat37', 'dst_feat38', 'dst_feat39', 'dst_feat40', 'dst_feat41', 'dst_feat42', 'dst_feat43', 'dst_feat44', 'dst_feat45', 'dst_feat46', 'dst_feat47', 'dst_feat48']


Number of Fourier features: 


49


Fourier operator: 
[[ 0.12558104  0.25066647  0.37476263 ...  0.37476263  0.25066647
   0.12558104]
 [ 0.25066647  0.49737977  0.73624911 ... -0.73624911 -0.49737977
  -0.25066647]
 [ 0.37476263  0.73624911  1.07165359 ...  1.07165359  0

In [6]:
#a = np.array([1+1j,2+3j,4+8j])
#np.imag(a)

In [7]:
#plt.plot(np.imag(simulation.fourier_operator[:,0]))

In [8]:
#np.sin(np.pi/(50))*2

In [9]:
#2*np.sin(np.pi/50*49)

In [10]:
#simulation.dataframe.iloc[331456,:]

In [11]:
#for ix, obj in enumerate(simulation.dataframe.iloc[331456,:].values):
#    print(ix,obj)

In [12]:
#simulation.fourier_engineered_dataframe.dct_feat0

In [13]:
#simulation.features

In [14]:
#simulation.fourier_matrix

In [15]:
#1/simulation.n_features*simulation.dataframe.loc[:,simulation.features].sum(axis=1)

In [16]:
#df_2 = simulation.fourier_engineered_dataframe[simulation.fourier_engineered_dataframe.phase==2]
#df_2.describe()

In [17]:
#df_1 = simulation.fourier_engineered_dataframe[simulation.fourier_engineered_dataframe.phase==1]
#df_1.describe()

In [18]:
#df_0 = simulation.fourier_engineered_dataframe[simulation.fourier_engineered_dataframe.phase==0]
#df_0.describe()

In [19]:
#plt.figure(figsize=(10,10))
#plt.scatter(df_1.fourier_feat0,df_1.fourier_feat2)
#plt.scatter(df_0.fourier_feat0,df_0.fourier_feat2,alpha=0.4)

In [20]:
#plt.scatter(df_0.fourier_feat0,df_0.fourier_feat2,alpha=0.4)

In [21]:
#plt.scatter(df_2.fourier_feat0,df_2.fourier_feat2)

In [22]:
#df_1.hist(figsize=(15,15))

In [23]:
#df_0.hist(figsize=(15,15))

#### Checking initialization

In [24]:
%%time
n_features = simulation.n_features
n_hamiltonians = simulation.n_hamiltonians
n_ts = simulation.n_ts

print("n_features: ", n_features)
print("n_hamiltonians: ", n_hamiltonians)
print("n_ts: ", n_ts)

n_features:  100
n_hamiltonians:  6561
n_ts:  2
CPU times: user 286 µs, sys: 41 µs, total: 327 µs
Wall time: 181 µs


In [25]:
%%time
n_total = len(simulation.dataframe)
n_train = len(simulation.dataframe[simulation.dataframe.type_of == "train"])
n_val = len(simulation.dataframe[simulation.dataframe.type_of == "val"])
n_test = len(simulation.dataframe[simulation.dataframe.type_of == "test"])
n_train_hamiltonians = len(simulation.train_ids)
n_val_hamiltonians = len(simulation.val_ids)
n_test_hamiltonians = len(simulation.test_ids)
n_total_hamiltonians = n_train_hamiltonians + n_val_hamiltonians + n_test_hamiltonians

print("% train: ", n_train/n_total)
print("% val: ",  n_val/n_total)
print("% test: ", n_test/n_total)
print("% train + val + test: ", (n_train+n_val+n_test)/n_total)
print("\n")
print("number of train hamiltonians: ", n_train_hamiltonians)
print("number of val hamiltonians: ", n_val_hamiltonians)
print("number of test hamiltonians: ", n_test_hamiltonians)
print("total number of hamiltonians: ", n_total_hamiltonians)
print("\n")
print("train ids: ", simulation.train_ids)
print("val ids: ", simulation.val_ids)
print("test ids: ", simulation.test_ids)

% train:  0.846822130772748
% val:  0.0
% test:  0.15317786922725193
% train + val + test:  1.0


number of train hamiltonians:  5556
number of val hamiltonians:  0
number of test hamiltonians:  1005
total number of hamiltonians:  6561


train ids:  [9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201

## Running a simulation

In [26]:

%%time
simulation.run_simulation(n_experiments, start_n, fit_params,shuffle_rows, pred_params, random_features, \
                          store_in_lists, save_eigenvector, save_hamiltonian, save_accuracy,\
                          save_models)
if simulation.fourier_mode is not None:
    simulation.fourier_dataframe.head(10)

running experiments:   0%|          | 0/5 [00:00<?, ?it/s]

Simulation mode:  dst


running experiments: 100%|██████████| 5/5 [00:06<00:00,  1.23s/it]

CPU times: user 6.04 s, sys: 97.4 ms, total: 6.13 s
Wall time: 6.13 s


In [27]:
simulation.dataframe.head(10)

,id,path,t1,t2,winding,phase,pred_phase,type_of,feat0,feat1,...,feat90,feat91,feat92,feat93,feat94,feat95,feat96,feat97,feat98,feat99
0,0,H_-2_-2_0.509296.dat,-2.0,-2.0,0.509296,999,666,test,0.100000,0.100000,...,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000
1,0,H_-2_-2_0.509296.dat,-2.0,-2.0,0.509296,999,666,test,0.141421,0.141142,...,0.114412,0.108967,0.123928,0.119406,0.131490,0.127962,0.136978,0.134500,0.140306,0.138916
2,0,H_-2_-2_0.509296.dat,-2.0,-2.0,0.509296,999,666,test,0.141421,0.140306,...,0.043702,0.026500,0.075777,0.060214,0.103092,0.090145,0.123928,0.114412,0.136978,0.131490
3,0,H_-2_-2_0.509296.dat,-2.0,-2.0,0.509296,999,666,test,-0.100000,0.100000,...,-0.100000,0.100000,-0.100000,0.100000,-0.100000,0.100000,-0.100000,0.100000,-0.100000,0.100000
4,0,H_-2_-2_0.509296.dat,-2.0,-2.0,0.509296,999,666,test,0.000509,-0.008372,...,-0.082713,-0.089753,-0.067684,-0.075347,-0.051587,-0.059753,-0.034677,-0.043217,-0.017220,-0.026000
5,0,H_-2_-2_0.509296.dat,-2.0,-2.0,0.509296,999,666,test,0.001204,-0.016530,...,-0.134123,-0.138686,-0.118757,-0.127445,-0.095929,-0.108196,-0.067073,-0.082149,-0.034003,-0.050940
6,0,H_-2_-2_0.509296.dat,-2.0,-2.0,0.509296,999,666,test,-0.141421,-0.138980,...,0.043375,0.067830,-0.009222,0.017385,-0.060524,-0.035502,-0.103326,-0.083403,-0.131616,-0.119589
7,0,H_-2_-2_0.509296.dat,-2.0,-2.0,0.509296,999,666,test,-0.000826,0.025688,...,0.134753,0.124324,0.141088,0.140407,0.127608,0.136771,0.096206,0.113925,0.051292,0.075079
8,0,H_-2_-2_0.509296.dat,-2.0,-2.0,0.509296,999,666,test,0.141417,0.136684,...,-0.115093,-0.131915,-0.061266,-0.091040,0.007717,-0.027643,0.074791,0.042592,0.123363,0.102291
9,0,H_-2_-2_0.509296.dat,-2.0,-2.0,0.509296,999,666,test,0.001302,0.036430,...,0.082068,0.050848,0.127402,0.108132,0.141218,0.138667,0.120099,0.134896,0.069268,0.097755


#### Viewing a random sample

In [28]:
%%time
if simulation.fourier_mode is not None:
    simulation.fourier_dataframe.sample(frac=0.1, replace=False)

CPU times: user 66.9 ms, sys: 0 ns, total: 66.9 ms
Wall time: 66.1 ms


#### Checking train/val/test splits again

In [29]:
%%time
n_total = len(simulation.dataframe)
n_train = len(simulation.dataframe[simulation.dataframe.type_of == "train"])
n_val = len(simulation.dataframe[simulation.dataframe.type_of == "val"])
n_test = len(simulation.dataframe[simulation.dataframe.type_of == "test"])
n_train_hamiltonians = len(simulation.train_ids)
n_val_hamiltonians = len(simulation.val_ids)
n_test_hamiltonians = len(simulation.test_ids)
n_total_hamiltonians = n_train_hamiltonians + n_val_hamiltonians + n_test_hamiltonians

print("% train: ", n_train/n_total)
print("% val: ",  n_val/n_total)
print("% test: ", n_test/n_total)
print("% train + val + test: ", (n_train+n_val+n_test)/n_total)
print("\n")
print("number of train hamiltonians: ", n_train_hamiltonians)
print("number of val hamiltonians: ", n_val_hamiltonians)
print("number of test hamiltonians: ", n_test_hamiltonians)
print("total number of hamiltonians: ", n_total_hamiltonians)
print("\n")
print("train ids: ", simulation.train_ids)
print("val ids: ", simulation.val_ids)
print("test ids: ", simulation.test_ids)

% train:  0.08474317939338516
% val:  0.7620789513793629
% test:  0.15317786922725193
% train + val + test:  1.0


number of train hamiltonians:  556
number of val hamiltonians:  5000
number of test hamiltonians:  1005
total number of hamiltonians:  6561


train ids:  [452, 1163, 3711, 2755, 5643, 1042, 3997, 300, 4574, 691, 2267, 6156, 4500, 4750, 4735, 2239, 1267, 472, 5323, 3919, 3789, 2058, 3185, 4113, 3688, 4910, 1658, 231, 955, 4353, 5615, 1370, 3899, 4456, 3013, 5926, 3466, 5729, 3395, 1766, 3038, 3842, 4744, 3909, 4519, 3960, 4866, 4901, 4823, 1916, 584, 1045, 2309, 1619, 1797, 1085, 3751, 3031, 5352, 781, 2386, 1574, 4334, 772, 6456, 3559, 5370, 5381, 4556, 2032, 2196, 3514, 5293, 2524, 4736, 4741, 2109, 939, 1901, 3359, 2598, 871, 3123, 1506, 5144, 1019, 1678, 5320, 5026, 6440, 3965, 2851, 5078, 1204, 5835, 2364, 62, 4045, 4395, 4212, 3758, 1443, 1415, 3748, 1419, 53, 3338, 5936, 2734, 3867, 5236, 5148, 4734, 1764, 60, 2259, 265, 4248, 1683, 6132, 5094, 2017, 1793, 5068, 2762

#### Checking summaries

In [30]:
%%time
ham_summary = simulation.hamiltonian_summary
print("length of ham_summary: ", len(ham_summary))
ham_summary

length of ham_summary:  6561
CPU times: user 128 µs, sys: 11 µs, total: 139 µs
Wall time: 102 µs


,t1,t2,type_of,1,phase,pred_phase
id,,,,,,
0,-2.0,-2.00,test,1.0,999,0
1,-2.0,-1.95,test,1.0,0,0
2,-2.0,-1.90,test,1.0,0,0
3,-2.0,-1.85,test,1.0,0,0
4,-2.0,-1.80,test,1.0,0,0
...,...,...,...,...,...,...
6556,2.0,1.80,test,1.0,0,0
6557,2.0,1.85,test,1.0,0,0
6558,2.0,1.90,test,1.0,0,0


In [31]:
ham_summary.describe()

,t1,t2,1,phase,pred_phase
count,6561.000000,6.561000e+03,6561.0,6561.000000,6561.0
mean,0.000000,-4.331917e-18,1.0,25.002134,0.0
std,1.169134,1.169134e+00,0.0,154.495657,0.0
min,-2.000000,-2.000000e+00,1.0,0.000000,0.0
25%,-1.000000,-1.000000e+00,1.0,0.000000,0.0
50%,0.000000,0.000000e+00,1.0,1.000000,0.0
75%,1.000000,1.000000e+00,1.0,1.000000,0.0
max,2.000000,2.000000e+00,1.0,999.000000,0.0


In [32]:
eigen_summary = simulation.eigenvector_summary
print("length of ham_summary: ", len(eigen_summary))
eigen_summary

length of ham_summary:  656100


,id,phase,pred_phase,type_of
0,0,999,1,test
1,0,999,1,test
2,0,999,1,test
3,0,999,1,test
4,0,999,1,test
...,...,...,...,...
656095,6560,999,1,test
656096,6560,999,1,test
656097,6560,999,1,test
656098,6560,999,1,test


#### Checking accuracies

In [33]:
simulation.accuracy

{'eigenvector_train': 0.5089928057553957,
 'eigenvector_val': 0.4902,
 'eigenvector_test': 0.5521327014218009,
 'hamiltonian_train': 0.4910071942446043,
 'hamiltonian_val': 0.5098,
 'hamiltonian_test': 0.44786729857819907}

#### Checking data stored in  memory

In [34]:
ham_summary_list = simulation.hamiltonian_summary_list
ham_summary_list

[]

In [35]:
eigen_summary_list = simulation.eigenvector_summary_list
eigen_summary_list

[]

In [36]:
accuracy_list = simulation.accuracy_list
accuracy_list

{'eigenvector_train': [],
 'eigenvector_val': [],
 'eigenvector_test': [],
 'hamiltonian_train': [],
 'hamiltonian_val': [],
 'hamiltonian_test': []}

In [37]:
#simulation.fourier_matrix

In [38]:
#simulation.fourier_matrix.shape

In [39]:
#simulation.fourier_matrix[:,0]

In [40]:
#simulation.fourier_matrix[:,1]

In [41]:
#np.exp(-1j*2*np.pi*3/100)

In [42]:
#for i in range(1,50):
#    print("\n")
#    print(i)
#    print(np.sum((np.real(simulation.fourier_matrix[:,i]) - np.real(simulation.fourier_matrix[:,-i]))**2))
#    print(np.sum((np.imag(simulation.fourier_matrix[:,i])+ np.imag(simulation.fourier_matrix[:,-i]))**2))

In [43]:
#print(simulation.fourier_matrix[:,0])

In [44]:
#print(simulation.fourier_matrix[:,50])